Import library yang digunakan

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from num2words import num2words
import re
import string
import emoji
import json

In [5]:
df = pd.read_json('sentimen_analisis.json')
df.head()

,text,author,emoji,cleantext,terbilang,tokenized,delangka,normalisasi,stemmed,polarity,label
0,"Turunkan tunjangan untuk DPR, alihkan untuk su...",abu studio chanel,"Turunkan tunjangan untuk DPR, alihkan untuk su...",turunkan tunjangan untuk dpr alihkan untuk sub...,turunkan tunjangan untuk dpr alihkan untuk sub...,"[turunkan, tunjangan, untuk, dpr, alihkan, unt...",turunkan tunjangan untuk dpr alihkan untuk sub...,"[turunkan, tunjangan, untuk, dpr, alihkan, unt...","[turun, tunjang, untuk, dpr, alih, untuk, subs...",-0.0024,negatif
1,Terimakasih pak de Saya seorang tukang ojek ka...,al rozaq,Terimakasih pak de Saya seorang tukang ojek ka...,terimakasih pak de saya seorang tukang ojek ka...,terimakasih pak de saya seorang tukang ojek ka...,"[terimakasih, pak, de, saya, seorang, tukang, ...",terimakasih pak de saya seorang tukang ojek ka...,"[terimakasih, bapak, de, saya, seorang, tukang...","[terimakasih, bapak, de, saya, orang, tukang, ...",-0.0011,negatif
2,Saya sebagai rakyat kecil mendengar bbm naik s...,Davin Davin,Saya sebagai rakyat kecil mendengar bbm naik s...,saya sebagai rakyat kecil mendengar bbm naik s...,saya sebagai rakyat kecil mendengar bbm naik s...,"[saya, sebagai, rakyat, kecil, mendengar, bbm,...",saya sebagai rakyat kecil mendengar bbm naik s...,"[saya, sebagai, rakyat, kecil, mendengar, bbm,...","[saya, bagai, rakyat, kecil, dengar, bbm, naik...",0.0000,positif
3,Terima kasih banyak Pak Jokowi. Kebijakan Bapa...,Zul Haqqi,Terima kasih banyak Pak Jokowi. Kebijakan Bapa...,terima kasih banyak pak jokowi kebijakan bapak...,terima kasih banyak pak jokowi kebijakan bapak...,"[terima, kasih, banyak, pak, jokowi, kebijakan...",terima kasih banyak pak jokowi kebijakan bapak...,"[terima, kasih, banyak, bapak, jokowi, kebijak...","[terima, kasih, banyak, bapak, jokowi, bijak, ...",0.0090,positif
4,Semoga Allah SWT memudahkan semua urusan rakya...,R.E. Handhita,Semoga Allah SWT memudahkan semua urusan rakya...,semoga allah swt memudahkan semua urusan rakya...,semoga allah swt memudahkan semua urusan rakya...,"[semoga, allah, swt, memudahkan, semua, urusan...",semoga allah swt memudahkan semua urusan rakya...,"[semoga, allah, swt, memudahkan, semua, urusan...","[moga, allah, swt, mudah, semua, urus, rakyat,...",0.0282,positif


In [9]:
import re
import string

def translate_emoticon(text):
        load_emoji = None
        with open('emoji/emoticon.json') as f:
            load_emoji = json.load(f)
        for index, keys in enumerate(load_emoji.keys()):
            text = text.replace(keys, load_emoji[keys])
        return text

def convert_emoji(text):
        text = re.sub('@[^\s]+','', text)
        text = re.sub('#[^\s]+','', text)
        text = emoji.demojize(text)
        text = text.replace(':', ' ')
        text = ' '.join(text.split())
        return text

def translate_emoji(text):
        df_emoji = pd.read_csv('emoji/emoji.csv')
        for index, row in df_emoji.iterrows():
            text = text.replace(row[1].replace(' ', '_'), row[5])
        return text

def clean(text):
        text = text.lower().strip() #Ubah Ke Lowercase
        text = re.sub('@[^\s]+','',text) #Menghapus Username
        text = ' '.join(re.sub("(rt )"," ", text).split()) #Menghapus kata 'rt'
        text = re.sub('((www\S+)|(http\S+))', ' ', text) #Menghapus URL
        text = text.encode('ascii', 'replace').decode('ascii') #remove non ASCII (emoticon, chinese word, .etc)
        text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"") #remove tab, new line, ans back slice
        text = text.translate(str.maketrans('','',string.punctuation)) #Menghapus Punctuation
        return text

def terbilang(text):
        kata = text.split()
        hasil =[]
        k = 0
        for i in kata:
                k = k + 1
                if i.isdigit():
                        h = num2words(int(i), lang='id').split()
                        hasil.extend(h)
                        continue
                hasil.extend([i])
        return ' '.join(hasil)

def delangka(text):
      text = re.sub(r'\d+', ' ', text)
      return text

def tokenize(column):
    """Tokenizes a Pandas dataframe column and returns a list of tokens.

    Args:
        column: Pandas dataframe column (i.e. df['text']).

    Returns:
        tokens (list): Tokenized list, i.e. [Donald, Trump, tweets]
    """

    tokens = nltk.word_tokenize(column)
    return [w for w in tokens if w.isalpha()]  

def load_dataset_normalization():
        load_word = pd.read_csv('normalisasi/normalisasi2.csv')
        normal_word_dict = {}
        for index, row in load_word.iterrows():
            if row[0] not in normal_word_dict:
                normal_word_dict[row[0]] = row[1]
        return normal_word_dict

def normalisasi(data):
        word_dict = load_dataset_normalization()
        return [word_dict[term] if term in word_dict else term for term in data]

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(data):
        lengthData = len(data)
        base = []
        count = 0
        for list_text in data:
            for text in list_text:
                base.append(stemmer.stem(text))
            count = count + 1
        return base



In [4]:
df["emoji"] = df["text"].apply(translate_emoticon)
df["emoji"] = df["emoji"].apply(convert_emoji)
df["emoji"] = df["emoji"].apply(translate_emoji)

NameError: name 'translate_emoticon' is not defined

In [6]:
df["cleantext"] = df["emoji"].apply(clean)

In [7]:
df["terbilang"] = df["cleantext"].apply(terbilang)

In [10]:
df["delangka"] = df['terbilang'].apply(delangka)

In [11]:
df['tokenized'] = df.apply(lambda x: tokenize(x['delangka']), axis=1)

In [13]:
df["normalisasi"] = df["tokenized"].apply(normalisasi)

In [14]:
hitung=0
listku = [[] for i in range(0, len(df["normalisasi"]))]
for list_text in df['normalisasi']:
    for text in list_text:
        listku[hitung].append(stemmer.stem(text))
    hitung = hitung + 1


df["stemmed"] = listku

In [15]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import json
import reprlib

# Memanfaatkan nltk VADER untuk menggunakan leksikon kustom
sia1A, sia1B, sia2 = SentimentIntensityAnalyzer(), SentimentIntensityAnalyzer(), SentimentIntensityAnalyzer()
# membersihkan leksikon VADER default
sia1A.lexicon.clear()
sia1B.lexicon.clear()
sia2.lexicon.clear()

# Membaca leksikon InSet
# Leksikon InSet lexicon dibagi menjadi dua, yakni polaritas negatif dan polaritas positif;
# kita akan menggunakan nilai compound saja untuk memberi label pada suatu kalimat
with open('leksikon/inset/_json_inset-neg.txt') as f:
    data1A = f.read()
with open('leksikon/inset/_json_inset-pos.txt') as f:
    data1B = f.read()
# Membaca leksikon sentiwords_id
with open('leksikon/sentistrength_id/_json_sentiwords_id.txt') as f:
    data2 = f.read()


# Mengubah leksikon sebagai dictionary
insetNeg = json.loads(data1A)
insetPos = json.loads(data1B)
senti = json.loads(data2)

# Update leksikon VADER yang sudah 'dimodifikasi'
sia1A.lexicon.update(insetNeg)
sia1B.lexicon.update(insetPos)
sia2.lexicon.update(senti)

In [16]:
def is_positive_inset(tweet: str):
    """True if tweet has positive compound sentiment, False otherwise."""
    return sia1A.polarity_scores(tweet)["compound"] + sia1B.polarity_scores(tweet)["compound"]

def is_positive_senti(tweet: str):
    """True if tweet has positive compound sentiment, False otherwise."""
    return sia2.polarity_scores(tweet)["compound"]

In [17]:
polarity = []
dataku = df['stemmed']
for komen in dataku:
        polarity.append(is_positive_inset(' '.join(komen)))

In [19]:
labels = []
for i in polarity:
    if i >= 0:
        labels.append('positif')
    else:
        labels.append('negatif')


df["polarity"] = polarity
df["label"] = labels

In [24]:
df.to_json("sentimen_analisis.json", orient='records')

In [23]:
df.groupby(["label"]).count()

,text,author,emoji,cleantext,terbilang,tokenized,delangka,normalisasi,stemmed,polarity
label,,,,,,,,,,
negatif,2041,2041,2041,2041,2041,2041,2041,2041,2041,2041
positif,959,959,959,959,959,959,959,959,959,959


In [25]:
df['delangka'].to_csv('data_train_fasttext.txt')